# Topic IV: Shrinkage and Variable Selection

**Information:**  
We are using the book 'G. James et al. -  An Introduction to Statistical Learning (with Applications in Python)'. You can find a copy of it for free [here](https://www.statlearning.com/).

In this exercise, we will predict the number of applications received using the other variables in the `College` data set.

## Import modules, packages and libraries

First, we import some useful modules, packages and libraries. These are needed for carrying out the computations and for plotting the results.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 

# sci-kit learn specifics
# We will use the sklearn package to obtain ridge regression and lasso models.
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import Ridge, Lasso
from sklearn.metrics import mean_squared_error

## Load the `College` data set

In [ ]:
college = pd.read_csv('College10.csv', index_col = 0)

# Display information about the data set
# college.info()

# Return summary statistics for each column
# college.describe()

# Return first five rows of the data set
college.head()

In [ ]:
### PREPROCESSING HERE

**(a) Normalize the data and split it into a training set and a test set.**

In [ ]:
### YOUR CODE HERE

**(b) Fit a linear model using least squares on the training set, and report the test error obtained.**

In [ ]:
### YOUR CODE HERE

**(c) Fit a ridge regression model on the training set, with $ \lambda $ chosen by cross-validation. Report the test error obtained.**

In [ ]:
### YOUR CODE HERE

**(d) Fit a lasso model on the training set, with $ \lambda $ chosen by cross-validation. Report the test error obtained, along with the number of non-zero coefficient estimates.**

In [ ]:
### YOUR CODE HERE

**(g) Comment on the results obtained. How accurately can we predict the number of college applications received?**

In [ ]:
### YOUR CODE HERE

\### YOUR COMMENTS HERE